In [1]:
import Gpib
import time
import matplotlib
import matplotlib.pyplot as plt

import ipywidgets as widgets

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import numpy as np

%matplotlib inline


In [2]:
import smu as sm
import meas
inst = Gpib.Gpib(0, 2)
smu = sm.hp4142b(inst)


HEWLETT PACKARD,4142B,0,4.30



In [26]:

Ic_max=0.2

#2N3904 EBC pinout

S=0
G=2
D=4

channels=['X', 'G', 'Y', 'D']

def meas_nfet(Vgs=0.0, Vds_max=10, Ids_max=0.1, Igs_max=1e-3, P_max=1):    
    smu.connect([G,D])
    smu.voltage(G, Vgs, i_lim=Igs_max)
  
    smu.sweep_v(D, start=0.01, stop=Vds_max, i_lim=Ids_max, mode='log', p_lim=P_max)
    smu.write("MM2,2,4\n")

    smu.xe()
    time.sleep(0.5)
    #print(smu.errors())

    smu.zero()
    #print(smu.opstat())

    time.sleep(0.5)
    smu.disconnect()

    r=smu.readresult()
    #print("RESULT: %s" % str(r))
    d=smu.parseresult(r, channels)

    return d

def meas_vgs_ids(Vds=5, Vgs_min=0, Vgs_max=12, Ids_max=10e-3):
    smu.connect([G,D])
    smu.voltage(D, Vds, i_lim=Ids_max)
  
    smu.sweep_v(G, start=Vgs_min, stop=Vgs_max, i_lim=1e-3)
    smu.write("MM2,2,4\n")

    smu.xe()
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    #print("RESULT: %s" % str(r))
    d=smu.parseresult(r, channels)

    return d


def meas_idss(Vds=10.0, Vgs=0.0, Ids_max=1e-4):
    smu.connect([G,D])
    smu.voltage(G, Vgs, i_lim=1e-3)
    smu.voltage(D, Vds, i_lim=Ids_max)
    
    smu.write("MM1,2,4")
    smu.xe()
    smu.disconnect()
    r=smu.readresult()
    d=smu.parseresult(r, channels)
    return d['I_D'][0]

def meas_vgs_on(Ids=1e-3, Vds=10, Ids_max=0.15):
    smu.connect([D,G])
    smu.search(G, start=0, stop=10, rate=10, i_lim=100e-6)
    smu.sense_i(D, v=Vds, i_target=Ids, i_lim=1.2*Ids) 
    smu.search_timing(hold=10e-3, delay=10e-3)
    smu.search_cfg(op='FBpos', meas="searchVsenseVI") #ASM 1,4,5e-4
    #smu.search_cfg(op='ramp_gt', meas="searchVsenseVI", itime=1e-3) #ASM 1,4,5e-4
    
    smu.write("MM6")
    smu.xe()
    #time.sleep(2)
    print(smu.opstat())
    smu.zero()
    smu.disconnect()
    r=smu.readresult()
    #print(r)
    d=smu.parseresult(r, channels)
    return d['V_G'][0]


In [12]:
#smu.calibrate()
#smu.average(10)
print(channels)
print(meas_idss())
print(meas_vgs_on(Ids=10e-3, Vds=10))
print(meas_vgs_ids(Ids_max=0.1))

['X', 'G', 'Y', 'D']
IBSTA: Complete END
5.3092e-10
IBSTA: Complete END
['Not active', 'V_SRC', 'Not active', 'V_SRC', 'Not active', 'Not active', 'Not active', 'Not active']
IBSTA: Complete END
2.5456
IBSTA: Complete END
MEAS_ERR: TBI+01.1632E-09
MEAS_ERR: CDI+099.972E-03
MEAS_ERR: VBI+199.999E+99
{'V_G': [0.0, 0.244, 0.489, 0.734, 0.979, 1.224, 1.469, 1.714, 1.959, 2.204, 2.448, 2.693, 2.938, 3.183], 'I_G': [1.4726e-09, 1.323e-09, -1.5538e-09, -1.2504e-09, 1.0078e-09, -1.2808e-09, -1.7148e-09, -1.6952e-09, -1.2038e-09, 3.4592e-10, -1.302e-09, 6.5898e-10, -1.2448e-10], 'I_D': [5.4334e-10, 3.9006e-10, -5.9388e-10, -2.7592e-10, 5.7506e-10, 1.32e-09, 3.6494e-08, 9.3644e-07, 2.5376e-05, 0.00050842, 0.0046736, 0.021018, 0.056462]}


In [27]:
#Measure n JFET operating curves
data = []

results = []

for vgs in np.arange(2.5, 10, 0.5):
    d=meas_nfet(Vgs=float(vgs), Vds_max=5, Ids_max=1, P_max=5)
    print(vgs)
    ids=np.array(d['I_D'])
    igs=np.array(d['I_G'])
    vds=np.array(d['V_D'])
    results.append ( { 'Ids': ids,
                        'Igs': igs,
                        'Vds': vds,
                        'Vgs': vgs,
                      'part': '2N7000',
                     't': time.time() } )

    
print("READY.")


IBSTA: Complete END
2.5
IBSTA: Complete END
3.0
IBSTA: Complete END
3.5
IBSTA: Complete END
4.0
IBSTA: Complete END
4.5
IBSTA: Complete END
5.0
IBSTA: Complete END
5.5
IBSTA: Complete END
MEAS_ERR: XBI-013.948E-09
6.0
IBSTA: Complete END
MEAS_ERR: TBI-02.2994E-09
MEAS_ERR: VBI+199.999E+99
6.5
IBSTA: Complete END
MEAS_ERR: TBI+01.1532E-09
MEAS_ERR: CDI+0.99960E+00
MEAS_ERR: VBI+199.999E+99
7.0
IBSTA: Complete END
MEAS_ERR: TBI+01.1588E-09
MEAS_ERR: CDI+0.99966E+00
MEAS_ERR: VBI+199.999E+99
7.5
IBSTA: Complete END
MEAS_ERR: TBI+01.3452E-09
MEAS_ERR: CDI+0.99962E+00
MEAS_ERR: VBI+199.999E+99
8.0
IBSTA: Complete END
MEAS_ERR: TBI+01.0862E-09
MEAS_ERR: CDI+0.99962E+00
MEAS_ERR: VBI+199.999E+99
8.5
IBSTA: Complete END
MEAS_ERR: TBI-01.5194E-09
MEAS_ERR: CDI+0.99962E+00
MEAS_ERR: VBI+199.999E+99
9.0
IBSTA: Complete END
MEAS_ERR: TBI-01.3612E-09
MEAS_ERR: VBI+199.999E+99
9.5
READY.


In [29]:
data= []

for r in results:
    trace =  go.Scatter(x=r['Vds'], y=1e3*r['Ids'], name='Vgs=%.2f' % (r['Vgs']))
    data.append(trace)

layout = go.Layout(
    title='2N7000',
    xaxis=dict(
        title='V_ds(V)',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I_ds(mA)', 
        #range=[0, 300],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout })

## data= []

for r in results:
    trace =  go.Scatter(x=r['Vds'], y=1e3*r['Ids'], name='Vgs=%.2f' % (r['Vgs']))
    data.append(trace)

layout = go.Layout(
    title='2SK170',
    xaxis=dict(
        title='V_ds(mA)',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I_ds(mA)', 
        #range=[0, 300],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout })